# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
0,0,punta arenas,CL,41.11,70,75,18.41,-53.1500,-70.9167,1687027278
1,1,bilibino,RU,41.85,66,100,6.80,68.0546,166.4372,1687027370
2,2,elbistan,TR,69.67,54,0,5.95,38.2059,37.1983,1687027753
3,3,sur,OM,87.75,66,74,8.32,22.5667,59.5289,1687027372
4,4,bredasdorp,ZA,52.68,75,0,9.98,-34.5322,20.0403,1687027367


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [37]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Temperature"] < 90) & (city_data_df["Temperature"] > 70)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Windspeed"] < 20)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] < 30)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
6,6,ar rutbah,IQ,89.65,18,5,9.80,33.0381,40.2848,1687027753
7,7,holualoa,US,77.56,78,20,4.61,19.6228,-155.9522,1687027374
8,8,colonia,DE,81.82,33,0,3.44,50.9333,6.9500,1687027382
16,16,adamstown,PN,71.78,74,9,8.72,-25.0660,-130.1015,1687027368
23,23,afif,SA,86.83,9,0,12.01,23.9065,42.9172,1687027404
...,...,...,...,...,...,...,...,...,...,...
528,528,sechura,PE,83.34,56,0,13.24,-5.5569,-80.8222,1687027802
539,539,dauphin,CA,77.14,36,20,5.75,51.1494,-100.0502,1687027804
546,546,carutapera,BR,81.86,79,16,5.55,-1.1950,-46.0200,1687027397
550,550,mazagao,BR,80.65,62,18,4.81,-0.1150,-51.2894,1687027805


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Latitude"]
lng = city_data_df["Longitude"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[{"City": city,
                        "Country": country,
                        "Latitude": lat,
                        "Longitude": lng,
                        "Humidity": humidity}]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\stcar\AppData\Local\Temp\ipykernel_9868\274125230.py:9: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  hotel_df = city_data_df[{"City": city,
C:\Users\stcar\AppData\Local\Temp\ipykernel_9868\274125230.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Latitude,Longitude,Humidity,Hotel Name
6,ar rutbah,IQ,33.0381,40.2848,18,
7,holualoa,US,19.6228,-155.9522,78,
8,colonia,DE,50.9333,6.9500,33,
16,adamstown,PN,-25.0660,-130.1015,74,
23,afif,SA,23.9065,42.9172,9,
...,...,...,...,...,...,...
528,sechura,PE,-5.5569,-80.8222,56,
539,dauphin,CA,51.1494,-100.0502,36,
546,carutapera,BR,-1.1950,-46.0200,79,
550,mazagao,BR,-0.1150,-51.2894,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 10000
params = {
  "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Latitude"]
    lon = row["Longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ar rutbah - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
colonia - nearest hotel: Wasserturm Hotel Cologne
adamstown - nearest hotel: No hotel found
afif - nearest hotel: Samaya Royal Afif
ghadamis - nearest hotel: فندق عين الفارس
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
west island - nearest hotel: Cocos Village Bungalows
pacasmayo - nearest hotel: Playa Linda
port mathurin - nearest hotel: Escale Vacances
mabini - nearest hotel: Crystal Waves Hotel
abqaiq - nearest hotel: No hotel found
constantia - nearest hotel: Hotel Chérica
ribeira grande - nearest hotel: Hospedaria JSF - 3 Estrelas
al bardiyah - nearest hotel: فندق البردي
sayre - nearest hotel: Comfort Inn
acarau - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
malvan - nearest hotel: Hotel Silver Sand
saipan - nearest hotel: Chalan Kanoa Beach Hotel
jamestown - nearest hotel: DoubleTree Jamestown
arenapolis - nearest hotel: No hotel found
kemin - nearest 

,City,Country,Latitude,Longitude,Humidity,Hotel Name
6,ar rutbah,IQ,33.0381,40.2848,18,No hotel found
7,holualoa,US,19.6228,-155.9522,78,Kona Hotel
8,colonia,DE,50.9333,6.9500,33,Wasserturm Hotel Cologne
16,adamstown,PN,-25.0660,-130.1015,74,No hotel found
23,afif,SA,23.9065,42.9172,9,Samaya Royal Afif
...,...,...,...,...,...,...
528,sechura,PE,-5.5569,-80.8222,56,Hospedaje Claritza
539,dauphin,CA,51.1494,-100.0502,36,No hotel found
546,carutapera,BR,-1.1950,-46.0200,79,No hotel found
550,mazagao,BR,-0.1150,-51.2894,62,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Hotel Name,Country)